In [4]:
%pip install -Uq upgini catboost

In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv('salesforcast.csv')
data = data.sample(n = 19_000,random_state=0)
data['store'] = data['store'].astype(str)
data['item'] = data['item'].astype(str)


data['date'] = pd.to_datetime(data['date'])
data.sort_values('date',inplace = True)

data.reset_index(inplace = True,drop = True)
data.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [6]:
train = data[data['date']< '2017-01-01']
test = data[data['date']>= '2017-01-01']

In [7]:
train_features = train.drop(columns='sales')
train_target = train['sales']

test_features = test.drop(columns='sales')
test_target = test['sales']

# Enrich Feature

In [8]:
from upgini import FeaturesEnricher,SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date": SearchKey.DATE
    },
    cv = CVType.time_series

)
enricher.fit(train_features,
             train_target,
             eval_set = [(test_features,test_target)])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...


Running search request, search_id=a69d3d23-28f1-41f9-9a85-7f8cbbabb70d
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


24 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_umap_31_fa6d9a99,0.0394,100.0000,"5.4444, 4.6961, 5.1643",Upgini,Weather & climate normals data,Premium
f_events_date_week_sin1_847b5db1,0.0322,100.0000,"0.7818, 0.9749, -0.4339",Upgini,Calendar data,Free
f_weather_date_weather_umap_48_b39cd0c4,0.0304,100.0000,"4.8408, 5.7568, 4.8182",Upgini,Weather & climate normals data,Premium
f_events_date_year_cos1_9014a856,0.0300,100.0000,"0.9524, 0.0731, 0.4083",Upgini,Calendar data,Free
f_weather_date_weather_pca_0_d7e0a1fc,0.0124,100.0000,"28.8387, 28.7578, 31.7943",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_34_c3ef5b4f,0.0110,100.0000,"4.8041, 5.6718, 5.4653",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_47_5123ef0a,0.0096,100.0000,"5.2537, 5.314, 5.2857",Upgini,Weather & climate normals data,Premium
f_events_date_week_cos3_7525fe31,0.0090,100.0000,"-0.2225, -0.2225, -0.2225",Upgini,Calendar data,Free
f_weather_date_weather_umap_13_842ae1fc,0.0064,100.0000,"5.4898, 3.8434, 5.0048",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_33_89bb7578,0.0044,100.0000,"4.9347, 4.865, 5.0255",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_16_25ac8b87,0.0029,100.0000,"6.5319, 5.9726, 5.4969",Upgini,Weather & climate normals data,Premium


Upgini,Weather & climate normals data,0.1207,12
Upgini,Calendar data,0.0718,4
Upgini,Markets data,0.0052,3
Upgini,World economic indicators,0.0024,5


We detected 113 outliers in your sample.
Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 19000
After dropping target outliers size: 18887
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15148,49.9541,309.9905,190.8005,119.1900
Eval 1,3739,58.0011,509.2897,371.4551,137.8346


In [9]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric


model = CatBoostRegressor(verbose = False,allow_writing_files=False,random_state = 0)

enricher.calculate_metrics(
    train_features,train_target,
      eval_set = [(test_features,test_target)],
      estimator = model,
      scoring = "mean_absolute_percentage_error"

)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15148,49.9541,0.255109,0.153784,0.101325
1,Eval 1,3739,58.0011,0.270377,0.198137,0.072241


In [10]:
enriched_train_features = enricher.transform(train_features,keep_input = True)
enriched_train_features = enricher.transform(test_features,keep_input = True)

enriched_train_features.head()


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=061f0958-b88a-4764-babb-8e5603c4e6c1
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=48621c37-5704-4705-97b3-592d39340c46
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_events_date_week_sin1_847b5db1,f_events_date_year_cos1_9014a856,f_events_date_week_cos3_7525fe31,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,f_financial_date_nasdaq_7d_to_1y_b00bfaa7,f_financial_date_stoxx_043cbcd4,f_economic_date_cbpol_pca_2_33d6e3fc,f_economic_date_cbpol_pca_9_bde660b4,f_events_date_year_sin1_3c44bc64,f_economic_date_cbpol_umap_4_c5ce4e90,f_economic_date_cci_pca_3_10646e17,f_economic_date_cpi_umap_7_20d961e2
15213,2017-01-01,1,44,-0.781831,0.98522,-0.900969,1.13758,1.090387,361.420013,1.898013,-0.366616,0.171293,5.408277,4.14775,0.150985
15214,2017-01-01,4,25,-0.781831,0.98522,-0.900969,1.13758,1.090387,361.420013,1.898013,-0.366616,0.171293,5.408277,4.14775,0.150985
15215,2017-01-01,3,15,-0.781831,0.98522,-0.900969,1.13758,1.090387,361.420013,1.898013,-0.366616,0.171293,5.408277,4.14775,0.150985
15216,2017-01-01,9,50,-0.781831,0.98522,-0.900969,1.13758,1.090387,361.420013,1.898013,-0.366616,0.171293,5.408277,4.14775,0.150985
15217,2017-01-01,4,37,-0.781831,0.98522,-0.900969,1.13758,1.090387,361.420013,1.898013,-0.366616,0.171293,5.408277,4.14775,0.150985


In [11]:
model.fit(train_features,train_target)
preds = model.predict(test_features)

eval_metric(test_target.values,preds,"SMAPE")

[37.65141857448004]